In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE3, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE3 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)})

In [5]:
# Find simmetries

In [6]:
%time adm_SE3 = SE3.find_admissible_projectivities()
len(adm_SE3)

CPU times: user 580 ms, sys: 79 ms, total: 659 ms
Wall time: 2.17 s


108

In [7]:
%time simm_SE3 = SE3.find_simmetries(adm_SE3)
len(simm_SE3)

CPU times: user 299 ms, sys: 160 ms, total: 459 ms
Wall time: 534 ms


6

In [8]:
# Study how the simmetries permute the Eckardt points

In [9]:
Eck_perms = [Permutation(SE3.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE3]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(6, 'S3')

In [10]:
### geometric interpretation of the group of symmetries

In [11]:
## Action of the stabilizer of SE3 on the 27 lines:
Ga = perm_group_lines(SE3, simm_SE3)

## Action of the stabilizer of SE3 on the 45 tritangent planes:
Gb = perm_group_planes(SE3, simm_SE3)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
S3

Structure of the group Gb:
S3


In [12]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [10]:
# First generator fixes A, permutes C, F. Second permutes A -> C -> F -> A
gens = Eck_perms_group.gens_small()
indices = [Eck_perms.index(gen) for gen in gens]

In [11]:
# Study how the simmetries permute the 27 lines

In [12]:
lines_perms = [Permutation(SE3.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE3]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(6, 'S3')

In [13]:
gens_lines = [lines_perms[index] for index in indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('E1', 'G4', 'F24', 'F13', 'F34', 'F45'),
  ('F13', 'F24', 'G4', 'E1', 'E4', 'F16')],
 [(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
  (1,14,9)(2,10,19)(3,4,13)(5,17,21)(6,16,20)(7,22,8)(11,26,24)(12,25,23)(15,27,18)])

In [14]:
PermutationGroup(gens_lines).is_isomorphic(Eck_perms_group)

True

In [15]:
# Find possible conditions to obtain a subfamily with a larger simmetry group

In [16]:
%time SE3.find_conditions_for_subfamilies(adm_SE3, simm_SE3)

CPU times: user 279 ms, sys: 56.9 ms, total: 336 ms
Wall time: 1.96 s


[]